In [5]:
import sys, glob, os
import numpy as np
import scipy as sp
import pandas as pd
from csv import reader, writer
import joblib
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Activation, Input, Dense, Dropout, Conv1D, Conv1DTranspose, BatchNormalization
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error

In [6]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [22]:
cwd = '/content/drive/MyDrive/Daylight Autonomy/'

In [7]:
df = pd.read_csv("/content/drive/MyDrive/Daylight Autonomy/output_v0.9_singleOutput_FullOutput_dummy.txt",
                 sep = ";",names=range(500))
df.fillna(value=0, inplace=True)

In [8]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,SideA,SideB,SideC,SideD,WindowWidth,WindowLength,Orientation,Area,IlluminancePerc,Illuminance,4,8.062258,2,8.062258,2.4,2.912473,0,24.0,0.640625,152,162,188,225,272,324,381,441,503,567,631,697,764,831,898,1000,89,106,140,186,240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SideA,SideB,SideC,SideD,WindowWidth,WindowLength,Orientation,Area,IlluminancePerc,Illuminance,3,8.000000,4,8.062258,1.8,2.191461,0,28.0,0.656250,128,155,194,242,296,352,410,470,531,592,654,716,779,842,905,1000,86,116,160,212,269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SideA,SideB,SideC,SideD,WindowWidth,WindowLength,Orientation,Area,IlluminancePerc,Illuminance,2,8.000000,4,8.246211,1.2,1.732772,0,24.0,0.648438,75,112,161,217,275,336,397,460,523,586,649,713,776,840,904,1000,49,88,140,197,257,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SideA,SideB,SideC,SideD,WindowWidth,WindowLength,Orientation,Area,IlluminancePerc,Illuminance,5,8.062258,4,8.000000,3.0,3.250000,0,36.0,0.693750,239,252,277,310,351,397,447,500,555,611,669,728,787,848,908,1000,173,190,220,260,307,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SideA,SideB,SideC,SideD,WindowWidth,WindowLength,Orientation,Area,IlluminancePerc,Illuminance,4,8.062258,4,8.062258,2.4,2.706012,0,32.0,0.664062,182,205,239,281,329,380,435,491,548,607,666,726,786,847,908,1000,126,152,191,238,291,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973,SideA,SideB,SideC,SideD,WindowWidth,WindowLength,Orientation,Area,IlluminancePerc,Illuminance,6,8.062258,3,8.246211,3.6,3.862642,0,36.0,0.703125,293,290,298,318,347,384,427,475,527,582,639,697,757,819,881,975,226,227,242,269,305,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2974,SideA,SideB,SideC,SideD,WindowWidth,WindowLength,Orientation,Area,IlluminancePerc,Illuminance,5,8.062258,3,8.062258,3.0,3.310589,0,32.0,0.675000,232,239,258,289,328,373,424,478,534,593,653,715,777,840,904,1000,165,177,203,241,288,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2975,SideA,SideB,SideC,SideD,WindowWidth,WindowLength,Orientation,Area,IlluminancePerc,Illuminance,4,8.062258,3,8.000000,2.4,2.778489,0,28.0,0.664062,167,184,214,254,302,354,410,468,527,588,650,713,776,839,903,1000,102,125,163,210,264,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2976,SideA,SideB,SideC,SideD,WindowWidth,WindowLength,Orientation,Area,IlluminancePerc,Illuminance,7,8.246211,3,8.246211,4.2,4.427189,0,40.0,0.750000,367,361,366,381,405,437,475,519,567,618,672,729,787,846,907,1000,298,297,307

In [13]:
headers = list(df.iloc[0,0:9])

data = df.iloc[:,10:18].copy()
data.columns = headers[0:8]

target = df.iloc[:,19:].copy()
target.columns = range(len(target.columns))

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=49)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=98)

In [21]:
class DNN():
    def __init__(self, node, layer, outpt):
        self.node = node
        self.layer = layer

        self.model = Sequential()

        for i in range(layer):
            self.model.add(Dense(node, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.001)))
            # model.add(BatchNormalization())
            self.model.add(Activation('relu'))
            # model.add(AlphaDropout(0.2))

        self.model.add(Dense(outpt, activation='linear'))

        optimizer = optimizers.Adam(learning_rate=0.0005, 
                                    beta_1=0.9, beta_2=0.999, amsgrad=True)

        self.model.compile(optimizer=optimizer, 
                          loss='mse')

        self.es = EarlyStopping(monitor='val_loss',
                               patience=300,
                               verbose=1)

        self.checkpoint_filepath = 'DNN.ckpt'.format(node, layer)

        self.mc = ModelCheckpoint(filepath = self.checkpoint_filepath,
                                 save_weights_only=True, 
                                 monitor='val_loss',
                                 save_best_only=True)
    def fit(self, X_train, y_train, X_val, y_val, epoch=3000):
        self.hist = self.model.fit(X_train, y_train, 
                                  epochs=epoch, 
                                  batch_size=1024, 
                                  verbose=2, 
                                  validation_data = (X_val, y_val),
                                  callbacks=[self.es, self.mc]);

    def visualize(self):
        loss = self.hist.history['loss']
        val_loss = self.hist.history['val_loss']

        fig = plt.figure()
        plt.plot(range(len(loss)), loss, label='loss')
        plt.plot(range(len(val_loss)), val_loss, label='val_loss')
        plt.xlabel('epochs')
        plt.ylabel('loss')
        plt.legend()
        plt.show()

    def predict(self, X):
        self.model.load_weights(self.checkpoint_filepath)
        return self.model.predict(X)

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)        
        return np.sqrt(mean_squared_error(y_test,y_pred))
        
    def print_eval(self, X_test, y_test):
        rmse = self.evaluate(X_test, y_test)
        
        print("Min RMSE: {:.2f}, Max RMSE: {:.2f}, Average RMSE: {:.2f}".format(min(rmse), max(rmse), np.mean(rmse)))

    def plot_eval(self, X_test, y_test):
        rmse, eep = self.evaluate(X_test, y_test)

        fig, axs = plt.subplots(figsize=(9,4));
        axs[0].hist(rmse, bins=100);
        axs[0].set_title('RMSE');

        # plt.savefig(os.path.join(cwd,'figures/hist_DNN_{}-{}.png'.format(self.node, self.layer)))

    def save(self, cwd, filename):
        json = os.path.join(cwd, 'models/{}_{}-{}.json'.format(filename, self.node, self.layer))
        hdf5 = os.path.join(cwd, 'models/{}_{}-{}.h5'.format(filename, self.node, self.layer))

        model_json = self.model.to_json()
        with open(json,'w') as json_file:
            json_file.write(model_json)
        self.model.save_weights(hdf5)

In [23]:
dnn = DNN(20, 3, len(target.columns))

In [25]:
dnn.fit(X_train, y_train, X_val, y_val, epoch=1000)

Epoch 1/1000
2/2 - 0s - loss: 41131.4922 - val_loss: 40567.3477
Epoch 2/1000
2/2 - 0s - loss: 40452.3672 - val_loss: 39935.9062
Epoch 3/1000
2/2 - 0s - loss: 39805.3047 - val_loss: 39334.0781
Epoch 4/1000
2/2 - 0s - loss: 39188.6758 - val_loss: 38761.5469
Epoch 5/1000
2/2 - 0s - loss: 38599.6211 - val_loss: 38218.3867
Epoch 6/1000
2/2 - 0s - loss: 38041.0742 - val_loss: 37704.0938
Epoch 7/1000
2/2 - 0s - loss: 37513.3242 - val_loss: 37218.0273
Epoch 8/1000
2/2 - 0s - loss: 37010.6641 - val_loss: 36759.8438
Epoch 9/1000
2/2 - 0s - loss: 36538.2109 - val_loss: 36328.7852
Epoch 10/1000
2/2 - 0s - loss: 36092.5781 - val_loss: 35923.7969
Epoch 11/1000
2/2 - 0s - loss: 35674.0273 - val_loss: 35544.2852
Epoch 12/1000
2/2 - 0s - loss: 35278.9609 - val_loss: 35189.2305
Epoch 13/1000
2/2 - 0s - loss: 34909.4688 - val_loss: 34857.5391
Epoch 14/1000
2/2 - 0s - loss: 34563.3867 - val_loss: 34548.1953
Epoch 15/1000
2/2 - 0s - loss: 34239.9062 - val_loss: 34259.9062
Epoch 16/1000
2/2 - 0s - loss: 339

In [26]:
dnn.evaluate(X_test, y_test)

161.73991758179017

In [29]:
y_pred = dnn.predict(X_test)

In [32]:
j = 35

print(X_test.iloc[j])
print("Simulated: {}".format(np.array(y_test.iloc[j])))
print("Predicted: {}".format(y_pred[j]))

SideA            4.000000
SideB            9.055385
SideC            4.000000
SideD            9.055385
WindowWidth      2.400000
WindowLength     2.753634
Orientation      0.000000
Area            36.000000
Name: 1057, dtype: float64
Simulated: [ 148.  168.  198.  236.  280.  326.  375.  426.  478.  531.  585.  639.
  694.  749.  805.  860.  916. 1000.   93.  116.  152.  195.  242.  293.
  345.  398.  452.  507.  562.  618.  674.  730.  786.  842.  899.  983.
   41.   70.  113.  161.  213.  267.  321.  377.  432.  488.  545.  601.
  658.  714.  771.  828.  885.  970.    2.   38.   87.  139.  194.  250.
  306.  362.  419.  475.  532.  589.  646.  703.  760.  817.  874.  960.
    0.   33.   81.  133.  188.  243.  299.  356.  412.  469.  526.  583.
  640.  697.  754.  811.  868.  953.   36.   58.   97.  144.  195.  248.
  302.  357.  413.  469.  525.  581.  638.  695.  751.  808.  865.  950.
   87.  101.  130.  169.  214.  263.  315.  367.  421.  475.  530.  586.
  641.  697.  753.  810.